In [138]:
import os
import json
import numpy
import pandas
import ast
import re
import datetime
import pickle

In [36]:
f = open("data/CrimeData/2011_01_assault_clustered.js", "r") 

In [37]:
lines = f.readlines()
f.close()

nHeaderLines = 3
nFooterLines = 3
records = len(lines) - nHeaderLines - nFooterLines


417

In [132]:
def parseLine(jsLine, crimeSet):
    cleanLine = jsLine.replace(" },\n", "")
    cleanLine = cleanLine.replace(" \"type\": \"Feature\", \"properties\": { \"Unnamed: 0\": ", "")
    cleanLine = cleanLine.split(",", 1)
    cleanLine = "{" + cleanLine[1]
    cleanLine = cleanLine.split(", \"geometry")
    cleanLine = cleanLine[0]
    cleanLine = re.sub("\"Offense 2\".*Location\"", "\"Location\"", cleanLine)
    #print(cleanLine)

    try:
        dictLine = ast.literal_eval(cleanLine)
        dictLine["DateTime"] = datetime.datetime.strptime(dictLine["Date"] + " " + dictLine["Time"], "%m\/%d\/%Y %I:%M %p")
        dictLine["CrimeSet"] = crimeSet
        return dictLine
    except ValueError:
        print("ERROR converting string")
        print(cleanLine)
        return "ERROR"


In [134]:
df = pandas.DataFrame()
path = "./data/CrimeData/"
allFiles = os.listdir(path)
nBadRecords = 0

for fileName in allFiles:
    print(fileName)
    f = open(path + fileName, "r") 
    lines = f.readlines()
    f.close()

    nHeaderLines = 3
    nFooterLines = 3
    crimeSet = fileName.split("_")[2]
    nRecords = len(lines) - nHeaderLines - nFooterLines
    print(nRecords)
    records = lines[nHeaderLines + 1:nHeaderLines + nRecords]
    iLine = nHeaderLines + 1
    for record in records:
        dictRecord = parseLine(record, crimeSet)
        if(dictRecord != "ERROR"):
            if(len(df.columns) == 0):
                df = pandas.DataFrame(dictRecord)
            else:
                df = df.append(dictRecord, ignore_index=True) 
        else:
            print("Error on line " + str(iLine))
            nBadRecords += 1;
        iLine += 1

2011_01_assault_clustered.js
411
2011_01_theft_clustered.js
713
2011_02_assault_clustered.js
375
2011_02_theft_clustered.js
603
2011_03_assault_clustered.js
466
2011_03_theft_clustered.js
935
2011_04_assault_clustered.js
438
2011_04_theft_clustered.js
942
2011_05_assault_clustered.js
483
2011_05_theft_clustered.js
922
2011_06_assault_clustered.js
467
2011_06_theft_clustered.js
876
2011_07_assault_clustered.js
423
2011_07_theft_clustered.js
953
2011_08_assault_clustered.js
410
2011_08_theft_clustered.js
805
2011_09_assault_clustered.js
357
2011_09_theft_clustered.js
848
2011_10_assault_clustered.js
364
2011_10_theft_clustered.js
960
2011_11_assault_clustered.js
382
2011_11_theft_clustered.js
840
2011_12_assault_clustered.js
391
2011_12_theft_clustered.js
800
2012_01_assault_clustered.js
438
2012_01_theft_clustered.js
697
2012_02_assault_clustered.js
413
2012_02_theft_clustered.js
680
2012_03_assault_clustered.js
464
2012_03_theft_clustered.js
816
2012_04_assault_clustered.js
458
2012_04

In [139]:
df.head()
pickleFile = open("df.pickle", "wb")
pickle.dump(df, pickleFile)
pickleFile.close()